<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/hr_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-8r8faq69
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-8r8faq69
  Resolved https://github.com/huggingface/transformers to commit 913d03dc5e78b82c24be7a52c9ad06dd1022f1e2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [47]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [48]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [50]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "hr", split="test")
#column_name = "audio, normalized_text"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [51]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

In [52]:
model = whisper.load_model("base")

100%|███████████████████████████████████████| 139M/139M [00:04<00:00, 32.4MiB/s]


In [53]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [54]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="hr")
    return result["text"]

In [55]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [56]:
from jiwer import wer
from collections import Counter

def calculate_precision_recall_f1(ground_truth, hypothesis):
    ground_truth_words = set(ground_truth.split())
    hypothesis_words = set(hypothesis.split())

    true_positives = ground_truth_words.intersection(hypothesis_words)
    false_positives = hypothesis_words - ground_truth_words
    false_negatives = ground_truth_words - hypothesis_words

    precision = len(true_positives) / (len(true_positives) + len(false_positives)) if true_positives or false_positives else 0
    recall = len(true_positives) / (len(true_positives) + len(false_negatives)) if true_positives or false_negatives else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall else 0

    return precision, recall, f1_score


In [57]:
total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
count = 0

for sample in voxpopuli_data:
    audio_file_path = sample['audio']['path']
    ground_truth_text = sample['normalized_text']

    # Transcribe the audio
    transcription = transcribe_audio(audio_file_path)

    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)

    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    count += 1

    print(f"Processed {count} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")
    # Calculate average metrics
average_wer = total_wer / count
average_precision = total_precision / count
average_recall = total_recall / count
average_f1 = total_f1 / count

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")


Processed 1 files, WER: 0.48214285714285715, Precision: 0.5098039215686274, Recall: 0.52, F1: 0.5148514851485149
Processed 2 files, WER: 1.0, Precision: 0.0, Recall: 0.0, F1: 0
Processed 3 files, WER: 0.5172413793103449, Precision: 0.45454545454545453, Recall: 0.4166666666666667, F1: 0.43478260869565216
Processed 4 files, WER: 0.8, Precision: 0.19230769230769232, Recall: 0.1724137931034483, F1: 0.18181818181818185
Processed 5 files, WER: 0.6666666666666666, Precision: 0.375, Recall: 0.3333333333333333, F1: 0.35294117647058826
Processed 6 files, WER: 0.717391304347826, Precision: 0.38636363636363635, Recall: 0.4146341463414634, F1: 0.39999999999999997
Processed 7 files, WER: 0.7894736842105263, Precision: 0.4444444444444444, Recall: 0.4444444444444444, F1: 0.4444444444444444
Processed 8 files, WER: 0.875, Precision: 0.21875, Recall: 0.18421052631578946, F1: 0.19999999999999998
Processed 9 files, WER: 0.8484848484848485, Precision: 0.25806451612903225, Recall: 0.27586206896551724, F1: 0.